# Ex.01: Precipitation at Sierra Nevada (Spain)

The first example is devoted to analyze the precipitation regime at XXX strong seasonality is observed. In order to guarantee that the model captures this temporal variability a Box-Cox transformation of input data will be done. Part of this example is included in [Cobos et al, 2021](link). As in example 01 (Precipitation at Sierra Nevada), the steps will be the following: 

1. Load marinetools module
2. Read the input data (observations, hindcasted, etc ...) and create the dictionary to fit the non-stationary probability model (PM)
3. Call marinetools.temporal.analysis.marginalfit for fitting the parameters of the PM
4. Create the dictionary for the multivariate and temporal dependency
5. Call marinetools.temporal.analysis.dependency to fit the parameters of the autoregressive model
6. Create the dictionary with the properties of the simulations
7. Call marinetools.temporal.simulation.simulation to generate new statistically equivalent time series
8. Call several plotting functions included marinetools.graphics to assess the goodness of fit

## 1. Load marinetools module

The following code load the basic functions (read data, analysis, simulation and plots) included in marinetools

In [ ]:
from marinetools.utils import read
from marinetools.temporal import analysis
from marinetools.temporal import simulation
from marinetools.graphics import plots

## 2. Read the input data and create the dictionary to make the marginal fit

For examples of reading files go to Ex01. The following code read the projections of freshwater river discharge from the RCP2.6 of the REMO2009 model in variable called "Qd". The input file is a xlsx file. As usually, some noise is included to ensure that the input variable is continuous and not discrete, which makes more difficult the analysis.

In [ ]:
data = read.xlsx("data/precipitacion")
data = data.loc[data["pr"] > 1]

Once the data is read, it is needed to create the dictionary with the properties about the temporal fluctuation and the probability models. In this example, a single PM given by a Weibull of maxima which parameter will be expanded in time using the sinusoidal structure and with 10 terms will be given. As it was depicted before, this example has the problem that high variability is found. In order to facilitate the convergence of the optimization, a previous Box-Cox transformation will be given. This information is translated to the dictionary as follow.

In [ ]:
params = {
    "pr": {
        "var": "pr",
        "non_stat_analysis": True,
        "basis_function": {"method": "trigonometric", "noterms": 5},
        "fun": {0: "weibull_max"},
        "transform": {"make": True, "plot": False, "method": "box-cox"},
        "ws_ps": [],
    }
}


## 3. Make the marginal fit
The following code will fit the parameters to the data.

In [ ]:
analysis.marginalfit(data, params)

The result will be saved at the file "Qd_weibull_max_nonst_1_sinusoidal_10_SLSQP.json" in the folder "marginalfit". Read carefully the information that the tool gives in the shell to understand that the parameters are passed correctly. You can also read the json file with any text editor and follows the results as the previous dictionary.

## 4. Create the dictionary for the multivariate and temporal dependency

The next step focus on the creation of the dictionary to the AR fit. In this case, it is quite normal to use the same parameters. In some cases, another model can be used.

## 8. Plots for verification

The following code will show the results from steps 3, 5 and 7. First all, the non-stationary cumulative distribution function is plotted. The following code shows how to represent the normalize (left panel) and not normalized (right panel) plots using the subplot command from matplotlib.pyplot.

In [ ]:
import matplotlib.pyplot as plt

params = read.rjson("marginalfit/Qd_weibull_max_nonst_1_sinusoidal_10_SLSQP")

fig, axs = plt.subplots(1, 2)
axs = axs.flatten()
params["transform"]["plot"] = True
plots.nonstationary_cdf(
    data,
    "Qd",
    params,
    date_axis=True,
    ax=axs[0],
    fname="to_axes",
)


params["transform"]["plot"] = False
plots.nonstationary_cdf(
    data,
    "Qd",
    params,
    date_axis=True,
    ax=axs[1],
)

![Ex02_NS_CDF.png][Image]

[Image]: Ex02_NS_CDF.png "Non-stationary CDF, normalize and not."